# Working with Galaxy objects

Synthesizer contains two types of galaxy:

- A particle `Galaxy` for working with galaxies comprised of individual stars, gas and/or black holes (`synthesizer.particle.galaxy.Galaxy`).
- A parametric `Galaxy` for working with parametric models described by SFZH and (when necessary) morphologies (`synthesizer.parametric.galaxy.Galaxy`).

As such instantiating and working with `Galaxy`s has the potential to lead to confusion. To simplify things for the user we provide a galaxy "factory function" which will always return the correct galaxy based on the arguments passed by the user while raising errors or warnings for incompatible combinations.

Below we set up the stellar populations we will need to demonstrate the use `Galaxy`s and the factory function. A description of the significance of the below operations can be found elsewhere in the documentation.

In [ ]:
import numpy as np
from synthesizer.grid import Grid
from synthesizer.parametric import SFH, ZDist
from synthesizer.parametric import Stars as ParametricStars
from synthesizer.particle.stars import sample_sfhz
from unyt import Myr

# Define the grid
grid_name = "test_grid"
grid_dir = "../../../tests/test_grid/"
grid = Grid(grid_name, grid_dir=grid_dir)

# Define the metallicity history
zh = ZDist.DeltaConstant(metallicity=0.01)

# Define the star formation history
sfh_p = {"duration": 100 * Myr}
sfh = SFH.Constant(duration=100 * Myr)

# Initialise the parametric Stars object
param_stars = ParametricStars(
    grid.log10age,
    grid.metallicity,
    sf_hist=sfh,
    metal_dist=zh,
    initial_mass=10**9,
)

# Define the number of stellar particles we want
n = 10000

# Sample the parametric SFZH, producing a particle Stars object
# we will also pass some keyword arguments for some example attributes
part_stars = sample_sfhz(
    sfzh=param_stars.sfzh,
    log10ages=param_stars.log10ages,
    log10metallicities=param_stars.log10metallicities,
    nstar=n,
    current_masses=np.full(n, 10**8.7 / n),
    redshift=1,
    initial_mass=10**6,
)

# Show what we've got out
print(type(param_stars))
print(type(part_stars))

## Creating a Galaxy

Now that we have the building blocks of both a particle and parametric galaxy we can import the factory function and get our galaxies. 

To do so we simply pass the factory function the arguments for the desired type of `Galaxy`. These are `Stars`, `Gas`, and `BlackHoles` objects from the `particle` and `parametric` modules respectively. In Synthesizer these different objects are called "components".

Note that both a particle and parametric `Galaxy` can be intialised with any combination of `Stars`, `Gas`, or `BlackHoles`. Each is a keyword argument which default to `None`.

In [ ]:
from synthesizer import galaxy

# Get a particle galaxy
part_gal = galaxy(stars=part_stars, gas=None, black_holes=None, redshift=1)
print(type(part_gal))

# Get a parametric galaxy
param_gal = galaxy(stars=param_stars, redshift=1)
print(type(param_gal))

Of course you are free to avoid this abstraction and explictly instantiate the desired `Galaxy`.

In [ ]:
from synthesizer.particle import Galaxy

# Get a particle galaxy
part_gal = Galaxy(stars=part_stars, gas=None, black_holes=None, redshift=1)

Or for a parametric galaxy.

In [ ]:
from synthesizer.parametric import Galaxy

# Get a parametric galaxy
param_gal = galaxy(stars=param_stars, redshift=1)

Once a galaxy has been created there are numerous methods for creating and plotting different types of spectra (e.g. [CAMELS example](../cosmo/galaxy_obj.ipynb)), making images or maps (e.g. [imaging docs](../imaging/particle_imaging.ipynb)), or computing properties. 